In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import itertools
import json
import random
import math
from google.colab import files
from google.colab import drive
import gspread
from google.auth import default

In [ ]:
!git clone https://github.com/felixludos/omni-data
!git clone https://github.com/felixludos/omni-belt
!pip install omni-belt/. omni-data/. omnifig
!apt-get -qq install -y graphviz && pip install pydot

Cloning into 'omni-data'...
remote: Enumerating objects: 1890, done.
remote: Counting objects: 100% (807/807), done.
remote: Compressing objects: 100% (276/276), done.
remote: Total 1890 (delta 571), reused 745 (delta 524), pack-reused 1083
Receiving objects: 100% (1890/1890), 1.26 MiB | 2.35 MiB/s, done.
Resolving deltas: 100% (1339/1339), done.
Cloning into 'omni-belt'...
remote: Enumerating objects: 1137, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1137 (delta 0), reused 1 (delta 0), pack-reused 1133
Receiving objects: 100% (1137/1137), 6.83 MiB | 11.17 MiB/s, done.
Resolving deltas: 100% (713/713), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./omni-belt
  Preparing metadata (setup.py) ... done
Processing ./omni-data
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 KB 811.6 kB/s eta 0:00:00
  Prepari

In [ ]:
from omnidata import hparam, inherit_hparams, submodule, submachine, spaces
from omnidata import Guru, Context, material, space, indicator, machine, Structured

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Fatal Python error: config_init_hash_seed: PYTHONHASHSEED must be "random" or an integer in range [0; 4294967295]
Python runtime state: preinitialized



In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
#defining my worksheet
graph2story = gc.open('20230212_Elements_for_Data_Generation').worksheet('graph2story_finer')
graph2temp = gc.open('20230212_Elements_for_Data_Generation').worksheet('graph2temp_finer')
graph2query = gc.open('20230212_Elements_for_Data_Generation').worksheet('query_dependencies')

#get_all_values gives a list of rows
dat1 = pd.DataFrame(graph2story.get_all_values())
dat2 = pd.DataFrame(graph2temp.get_all_values())
dat_query = pd.DataFrame(graph2query.get_all_values())
#Convert to a DataFrame 
dat1.columns = dat1.iloc[1]
dat2.columns = dat2.iloc[0]
dat_query.columns = dat_query.iloc[1]

# Remove the first two rows and reset the index
dat1 = dat1.iloc[2:].reset_index(drop=True)
dat2 = dat2.iloc[1:].reset_index(drop=True)
dat_query = dat_query.iloc[2:].reset_index(drop=True)

# Drop the empty column
dat1 = dat1.drop(columns=[''])
dat1.columns.name = None
dat2.columns.name = None
dat_query.columns.name = None
# Print the DataFrame


In [ ]:
merged_df = dat1.merge(dat2, on='causal_graph_type', how='left')
dat = merged_df.drop(columns = ['semantic_graph (by topological order)_y','variable_correspondence_y'])

dat['relation1'] = dat['relation1'].str.split(', ')
dat['relation2'] = dat['relation2'].str.split(', ')

dat.fillna('', inplace = True)

In [ ]:

dat_query['info_type'] = dat_query['info_type'].str.split(', ')
dat_query['causal_graph_type'] = dat_query['causal_graph_type'].str.split(', ')
dat_query.fillna('', inplace = True)

In [1]:
# diamond only

class Current(Structured): # Create a class for the current SCM
    binary_op = hparam('and')
    @material.from_size('X')
    def generate_X(self, N = 1):
    return np.random.choice([True, False], p=[0.5, 0.5]) # Bernoulli(0.5) prior

    @machine('Z')
    def Z(self, X):
    return X

    @machine('W')
    def W(self, X):
    return X

    @machine('Y')
    def Y(self, Z, W):
        return (Z and W) if self.binary_op == 'and' else (Z or W)

scms = {}

scms['firing_squad', 'diamond'] = Current(binary_op='or', application={'X':'captain', 'W': 'rifleman1', 'Z': 'rifleman2', 'Y': 'prisoner'})
scms['sprinkler', 'diamond'] = Current(binary_op='or', application={'X':'season', 'W': 'rain', 'Z': 'sprinkler', 'Y': 'wet_floor'})

with open('prop_diamond.json', 'w') as out_file:
  out_file.write("[\n")
  # Counter for number of entries written
  count = 0
  for idx, row in dat.iterrows():
    if row['causal_graph_type'] == 'diamond':

      # loop over 2 different Structural equations
      for rel1 in row['relation1']:

        # class Current(Structured): # Create a class for the current SCM
        #   @material.from_size('X')
        #   def generate_X(self, N = 1):
        #     return np.random.choice([True, False], p=[0.5, 0.5]) # Bernoulli(0.5) prior

        #   @machine('Z')
        #   def Z(self, X):
        #     return X

        #   @machine('W')
        #   def W(self, X):
        #     return X

        #   @machine('Y')
        #   def Y(self, Z, W):
        #     if rel1 == 'and':
        #       return Z and W
        #     else:
        #       return Z or W

        # Instantiate the SCM
        ctx = Guru(Current(binary_op=rel1))
        s = {'X','Z','W','Y'}
        powerset = set(frozenset(subset) for subset in itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1)))
        powerset = [set(subset) for subset in powerset]

        # loop over all sets of observed variables
        for obs_var in powerset:
          n = len(obs_var)
          # if nothing observed(n == 0) -> ask interventional questions only (do_calculus)
          if n == 0: 
            filled_template = row['given_info_word_template'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'], W_string=row['W_string_name'], 
                                                                      X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                      W_0 = row['W_0'], W_1 = row['W_1'], rel1 = rel1)
            
            # loop over all applicible queries (do_calculus only)
            for index, question in dat_query.iterrows():
              if row['causal_graph_type'] in question['causal_graph_type'] and row['info_type'] in question['info_type'] and question['CI_engine to get the answer'] == 'do_calculus':
                s2 = {'X','Z','W'}
                powerset2 = set(frozenset(subset) for subset in itertools.chain.from_iterable(itertools.combinations(s2, r) for r in range(len(s2)+1)))
                powerset2 = [set(subset) for subset in powerset2]
                for intervention_set in powerset2:
                  if len(intervention_set) > 0:
                    # loop over 2^n possible n-tuples
                    for intervention_val in list(itertools.product(*[(0, 1)] * len(intervention_set))):
                      interv = {}
                      interv_text = ""
                      # for each n-tuple, create a dictionary of interventions
                      for i, (k, v) in enumerate(zip(intervention_set, intervention_val)):

                        interv_text += row[f'{k}_{v}']
                        interv[k] = v
                        if i != len(intervention_set) - 1:
                            interv_text += ', '

                    for y in [0,1]:
                      filled_template = row['given_info_word_template'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'], W_string=row['W_string_name'], 
                                                                              X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                              W_0 = row['W_0'], W_1 = row['W_1'], rel1 = rel1)
                      query = question['query_word_form'].format(Z_string = row['Z_string_name'], Y_string=row['Y_string_name'], W_string=row['W_string_name'], 
                                                                            X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                            W_0 = row['W_0'], W_1 = row['W_1'], intervention = interv_text, Y_int = row[f'Y_{y}'])

                      # Perform interventions
                      for (var, val) in interv.items():
                        ctx[var] = val
                      
                      result = int(ctx['Y'] == y)
                      y_truth = int(ctx['Y'])
                      result_text = row[f'Y_{y_truth}']
                      target = 'Yes' if result else 'No'
                      

                      ctx.clear()
              
                      if count > 0:
                        out_file.write(",\n")
                            
                      data = {
                            "causal_graph_type": row['causal_graph_type'],
                            "causal_inference_engine": question['CI_engine to get the answer'],
                            "given_info_type": row['info_type'],
                            "input": filled_template + ' ' + query.replace('\n', ' '),
                            "target": target,
                            "truth": result_text
                        }
                      # Write data to file
                      json.dump(data, out_file, indent=4)
                      # Increment counter
                      count += 1

          # With observations, answer counterfactual questions.
          else:
            for obs_val in list(itertools.product(*[(0, 1)] * (n))):
              evid_text = ""
              evid = {}
              for i, (k, v) in enumerate(zip(obs_var, obs_val)):
                evid_text += row[f'{k}_{v}']
                evid[k] = v
                if i == len(obs_var) - 1:
                    evid_text += '.'
                else:
                    evid_text += ', '
              filled_template = row['given_info_word_template'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'], W_string=row['W_string_name'], 
                                                                      X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                      W_0 = row['W_0'], W_1 = row['W_1'], rel1 = rel1) +' '+\
              row['extra_info'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'], W_string=row['W_string_name'],X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                      W_0 = row['W_0'], W_1 = row['W_1'], rel1 = rel1, evidence = evid_text)
              for index, question in dat_query.iterrows():
                if row['causal_graph_type'] in question['causal_graph_type'] and row['info_type'] in question['info_type'] and question['CI_engine to get the answer'] == 'counterfactual':
                  for (cou_var, cou_val, y) in zip(['X','Z','W'],[0,1],[0,1]):
                    if cou_var not in evid or cou_val != evid[cou_var]:
                      query = question['query_word_form'].format(Z_string = row['Z_string_name'], Y_string=row['Y_string_name'], W_string=row['W_string_name'],
                                                                X_string = row['X_string_name'], X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                W_0 = row['W_0'], W_1 = row['W_1'], cou_var = row[f'{cou_var}_string_name'], cou_val = row[f'{cou_var}_{cou_val}'], Y_int = row[f'Y_{y}'], cou_val_original = row[f'{cou_var}_{abs(1-cou_val)}'])
                      
                      # Input observations into the SCM
                      for (var, val) in evid.items():
                        ctx[var] = val

                      # Input counterfactual by setting cou_var to cou_val
                      ctx[cou_var] = bool(cou_val)
                      result = int(ctx['Y'] == y)

                      # True state of Y in counterfactual world
                      y_truth = int(ctx['Y'])
                      result_text = row[f'Y_{y_truth}']
                      target = 'Yes' if result else 'No'
                      

                      ctx.clear()
                
                      if count > 0:
                        out_file.write(",\n")
                            
                      data = {
                            "causal_graph_type": row['causal_graph_type'],
                            "causal_inference_engine": question['CI_engine to get the answer'],
                            "given_info_type": row['info_type'],
                            "input": filled_template + ' ' + query.replace('\n', ' '),
                            "target": target,
                            "truth": result_text
                        }
                      # Write data to file
                      json.dump(data, out_file, indent=4)
                      # Increment counter
                      count += 1

                
              
  out_file.write("\n]")

out_file.close()

IndentationError: ignored

In [ ]:
# fork graph
with open('prop_fork.json', 'w') as out_file:
  out_file.write("[\n")
  # Counter for number of entries written
  count = 0
  for idx, row in dat.iterrows():
    if row['causal_graph_type'] == '>':
      # loop over 2 different Structural equations
      for rel1 in row['relation1']:
        class Current(Structured): # Create a class for the current SCM
          @material.from_size('X')
          def generate_X(self, N = 1):
            return np.random.choice([True, False], p=[0.5, 0.5]) # Bernoulli(0.5) prior

          @machine('Z')
          def Z(self, X):
            return X

          @machine('Y')
          def Y(self, X, Z):
            if rel1 == 'and':
              return X and Z
            else:
              return X or Z

        # Instantiate the SCM
        ctx = Guru(Current())
        s = {'X','Z','Y'}
        powerset = set(frozenset(subset) for subset in itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1)))
        powerset = [set(subset) for subset in powerset]

        # loop over all sets of observed variables
        for obs_var in powerset:
          n = len(obs_var)
          # if nothing observed(n == 0) -> ask interventional questions only (do_calculus)
          if n == 0: 
            filled_template = row['given_info_word_template'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                                                     X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                     rel1 = rel1)
            
            # loop over all applicible queries (do_calculus only)
            for index, question in dat_query.iterrows():
              if row['causal_graph_type'] in question['causal_graph_type'] and row['info_type'] in question['info_type'] and question['CI_engine to get the answer'] == 'do_calculus':
                s2 = {'X','Z'}      
                powerset2 = set(frozenset(subset) for subset in itertools.chain.from_iterable(itertools.combinations(s2, r) for r in range(len(s2)+1)))
                powerset2 = [set(subset) for subset in powerset2]
                for intervention_set in powerset:
                  if len(intervention_set) > 0:
                    # loop over 2^n possible n-tuples
                    for intervention_val in list(itertools.product(*[(0, 1)] * len(intervention_set))):
                      interv = {}
                      interv_text = ""
                      # for each n-tuple, create a dictionary of interventions
                      for i, (k, v) in enumerate(zip(intervention_set, intervention_val)):

                        interv_text += row[f'{k}_{v}']
                        interv[k] = v
                        if i != len(intervention_set) - 1:
                            interv_text += ', '

                      for y in [0,1]:
                        filled_template = row['given_info_word_template'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'], W_string=row['W_string_name'], 
                                                                                X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                                W_0 = row['W_0'], W_1 = row['W_1'], rel1 = rel1)
                        query = question['query_word_form'].format(Z_string = row['Z_string_name'], Y_string=row['Y_string_name'], W_string=row['W_string_name'], 
                                                                              X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                              W_0 = row['W_0'], W_1 = row['W_1'], intervention = interv_text, Y_int = row[f'Y_{y}'])

                        # Perform interventions
                        for (var, val) in interv.items():
                          ctx[var] = val
                        
                        result = int(ctx['Y'] == y)
                        y_truth = int(ctx['Y'])
                        result_text = row[f'Y_{y_truth}']
                        target = 'Yes' if result else 'No'
                        

                        ctx.clear()
                
                        if count > 0:
                          out_file.write(",\n")
                              
                        data = {
                              "causal_graph_type": row['causal_graph_type'],
                              "causal_inference_engine": question['CI_engine to get the answer'],
                              "given_info_type": row['info_type'],
                              "input": filled_template + ' ' + query.replace('\n', ' '),
                              "target": target,
                              "truth": result_text
                          }
                        # Write data to file
                        json.dump(data, out_file, indent=4)
                        # Increment counter
                        count += 1

          # With observations, answer counterfactual questions.
          else:
            for obs_val in list(itertools.product(*[(0, 1)] * (n))):
              evid_text = ""
              evid = {}
              for i, (k, v) in enumerate(zip(obs_var, obs_val)):
                evid_text += row[f'{k}_{v}']
                evid[k] = v
                if i == len(obs_var) - 1:
                    evid_text += '.'
                else:
                    evid_text += ', '
              filled_template = row['given_info_word_template'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],  
                                                                      X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                      rel1 = rel1) +' '+\
              row['extra_info'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                       X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                       rel1 = rel1, evidence = evid_text)

              for index, question in dat_query.iterrows():
                if row['causal_graph_type'] in question['causal_graph_type'] and row['info_type'] in question['info_type'] and question['CI_engine to get the answer'] == 'counterfactual':
                  for (cou_var, cou_val, y) in zip(['X','Z','W'],[0,1],[0,1]):
                    if cou_var not in evid or cou_val != evid[cou_var]:
                      query = question['query_word_form'].format(Z_string = row['Z_string_name'], Y_string=row['Y_string_name'],
                                                                X_string = row['X_string_name'], X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                cou_var = row[f'{cou_var}_string_name'], cou_val = row[f'{cou_var}_{cou_val}'], Y_int = row[f'Y_{y}'], cou_val_original = row[f'{cou_var}_{abs(1-cou_val)}'])
                      
                      # Input observations into the SCM
                      for (var, val) in evid.items():
                        ctx[var] = val

                      # Input counterfactual by setting cou_var to cou_val
                      ctx[cou_var] = bool(cou_val)
                      result = int(ctx['Y'] == y)

                      # True state of Y in counterfactual world
                      y_truth = int(ctx['Y'])
                      result_text = row[f'Y_{y_truth}']
                      target = 'Yes' if result else 'No'
                      

                      ctx.clear()
                
                      if count > 0:
                        out_file.write(",\n")
                            
                      data = {
                            "causal_graph_type": row['causal_graph_type'],
                            "causal_inference_engine": question['CI_engine to get the answer'],
                            "given_info_type": row['info_type'],
                            "input": filled_template + ' ' + query.replace('\n', ' '),
                            "target": target,
                            "truth": result_text
                        }
                      # Write data to file
                      json.dump(data, out_file, indent=4)
                      # Increment counter
                      count += 1

              
            

  out_file.write("\n]")
out_file.close()

In [ ]:
with open('prop_diamond.json', 'r') as f:
  unique_entries = {}
  duplicates = []
  data = json.load(f)
  print(f"Total number of data points in the file: {len(data)}")


  for entry in data:
      entry_hash = hash(json.dumps(entry, sort_keys=True))
      if entry_hash in unique_entries:
          duplicates.append(entry)
          print("Duplicate Entry:")
          print(json.dumps(entry, indent=2))
          print("Original Entry:")
          print(json.dumps(unique_entries[entry_hash], indent=2))
          print("-------------------")
      else:
          unique_entries[entry_hash] = entry

  if duplicates:
      print(f"There are {len(duplicates)} duplicates in the file.")
      print(f"Total number of data points in the file: {len(data)}")
      # Do something with the duplicates, such as printing them or removing them from the original data
  else:
      print("There are no duplicates in the file.")

out_file.close()

Total number of data points in the file: 960
There are no duplicates in the file.


In [ ]:
conf_nonid_df = dat[(dat['info_type'] == 'text_nonid') & (dat['causal_graph_type'] == 'confounder')]

In [ ]:
conf_nonid_df

,causal_graph_type,which_example,X_string_name,X_0,X_1,Z_string_name,Z_0,Z_1,Y_string_name,Y_0,...,semantic_graph (by topological order)_x,variable_correspondence_x,structural_equation,given_info_word_template,extra_info,suitable_CI_engines,,relation1,relation2,info_type
0,confounder,simpson_hospital,hospital assignment,treated at hospital A,treated at hospital B,age,young,old,recovery,not recover,...,"\nconf->cause, conf->effect, cause->effect","{X: cause, Y: effect, Z: conf}",,We know that {X_string} has an effect on {Y_st...,We observe that the probability of {Y_1} given...,"do_calculus; counterfactual_formal_solver,",,"[greater than, smaller than]",[],text_nonid
5,confounder,simpson_kidneystone,treatment assignment,receiving no treatment,receiving treatment,kidney stone size,small kidney stone,large kidney stone,recovery,not recover,...,"\nconf->cause, conf->effect, cause->effect","{X: cause, Y: effect, Z: conf}",,We know that {X_string} has an effect on {Y_st...,We observe that the probability of {Y_1} given...,"do_calculus; counterfactual_formal_solver,",,"[greater than, smaller than]",[],text_nonid
10,confounder,simpson_drug,drug assingment,no drug,drug,gender,non-male,male,recovery,not recover,...,"\nconf->cause, conf->effect, cause->effect","{X: cause, Y: effect, Z: conf}",,We know that {X_string} has an effect on {Y_st...,We observe that the probability of {Y_1} given...,"do_calculus; counterfactual_formal_solver,",,"[greater than, smaller than]",[],text_nonid


In [ ]:
# confounder graph with insufficient text info
with open('conf_nonid.json', 'w') as out_file:
  out_file.write("[\n")
  # Counter for number of entries written
  count = 0
  for idx, row in dat.iterrows():
    if row['causal_graph_type'] == 'confounder' and row['info_type'] == 'text_nonid':
      # loop over 2 different Structural equations
      for rel1 in row['relation1']:    
        filled_template = row['given_info_word_template'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                                                X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1']) +' '+\
                          row['extra_info'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                                  X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                  rel1 = rel1)
        
        # loop over all applicible queries with no observation (do_calculus only)
        for index, question in dat_query.iterrows():
          if 'confounder' in question['causal_graph_type'] and 'text_nonid' in question['info_type'] and question['CI_engine to get the answer'] == 'do_calculus': 
            if question['query_id'] == 'adj':
              query = question['query_word_form'].format(X_string = row['X_string_name'], Z_string = row['Z_string_name'], Y_string=row['Y_string_name'], W_string=row['W_string_name'], 
                                                                    X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                    W_0 = row['W_0'], W_1 = row['W_1'])

              target = row['Z_string_name']
              if count > 0:
                out_file.write(",\n")
                    
              data = {
                    "causal_graph_type": row['causal_graph_type'],
                    "causal_inference_engine": question['CI_engine to get the answer'],
                    "given_info_type": row['info_type'],
                    "input": filled_template + ' ' + query.replace('\n', ' '),
                    "target": target
                }

            else:
              query = question['query_word_form'].format(X_string = row['X_string_name'], Z_string = row['Z_string_name'], Y_string=row['Y_string_name'], W_string=row['W_string_name'], 
                                                                    X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                    W_0 = row['W_0'], W_1 = row['W_1'])

              target = 'cannot determine'
              if count > 0:
                out_file.write(",\n")
                    
              data = {
                    "causal_graph_type": row['causal_graph_type'],
                    "causal_inference_engine": question['CI_engine to get the answer'],
                    "given_info_type": row['info_type'],
                    "input": filled_template + ' ' + query.replace('\n', ' '),
                    "target": target
                }
            # Write data to file
            json.dump(data, out_file, indent=4)
            # Increment counter
            count += 1
          

          elif 'confounder' in question['causal_graph_type'] and 'text_nonid' in question['info_type'] and question['CI_engine to get the answer'] == 'counterfactual': 
            # With observations, answer counterfactual questions.
            s = {'X','Z','Y'}
            powerset = set(frozenset(subset) for subset in itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1)))
            powerset = [set(subset) for subset in powerset]
            filled_template = row['given_info_word_template'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                                              X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                              rel1 = rel1)+' '+\
                              row['extra_info'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                                      X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                      rel1 = rel1)


            for obs_set in powerset:
              n = len(obs_set)
              if n > 0:
                for obs_val in list(itertools.product(*[(0, 1)] * (n))):
                  evid_text = ""
                  evid = {}
                  for i, (k, v) in enumerate(zip(obs_set, obs_val)):
                    evid_text += row[f'{k}_{v}']
                    evid[k] = v
                    if i != len(obs_set) - 1:
                      evid_text += ', '
                        
                  for cou_val in [0,1]:
                    if 'X' not in evid or cou_val != evid['X']:
                      query = question['query_word_form'].format(X_string = row['X_string_name'],  Y_1 = row['Y_1'],
                                                                X_cou_val = row[f'X_{cou_val}'], val_original = row[f'X_{abs(1-cou_val)}'], evidence = evid_text)
            
                      if count > 0:
                        out_file.write(",\n")

                      target = 'cannot determine'
                            
                      data = {
                            "causal_graph_type": row['causal_graph_type'],
                            "causal_inference_engine": question['CI_engine to get the answer'],
                            "given_info_type": row['info_type'],
                            "input": filled_template + ' ' + query.replace('\n', ' '),
                            "target": target
                        }
                      # Write data to file
                      json.dump(data, out_file, indent=4)
                      # Increment counter
                      count += 1             
                  

  out_file.write("\n]")
out_file.close()


In [ ]:
conf_id_df = dat[(dat['info_type'] == 'text') & (dat['causal_graph_type'] == 'confounder')]

In [ ]:
conf_id_df

,causal_graph_type,which_example,X_string_name,X_0,X_1,Z_string_name,Z_0,Z_1,Y_string_name,Y_0,...,semantic_graph (by topological order)_x,variable_correspondence_x,structural_equation,given_info_word_template,extra_info,suitable_CI_engines,,relation1,relation2,info_type
1,confounder,simpson_hospital,hospital assignment,treated at hospital A,treated at hospital B,age,young,old,recovery,not recover,...,"\nconf->cause, conf->effect, cause->effect","{X: cause, Y: effect, Z: conf}",,We know that {X_string} has an effect on {Y_st...,We observe that the probability of {Y_1} given...,"do_calculus; counterfactual_formal_solver,",,"[greater than, smaller than]",[opposite of relation1],text
6,confounder,simpson_kidneystone,treatment assignment,receiving no treatment,receiving treatment,kidney stone size,small kidney stone,large kidney stone,recovery,not recover,...,"\nconf->cause, conf->effect, cause->effect","{X: cause, Y: effect, Z: conf}",,We know that {X_string} has an effect on {Y_st...,We observe that the probability of {Y_1} given...,"do_calculus; counterfactual_formal_solver,",,"[greater than, smaller than]",[opposite of relation1],text
11,confounder,simpson_drug,drug assingment,no drug,drug,gender,non-male,male,recovery,not recover,...,"\nconf->cause, conf->effect, cause->effect","{X: cause, Y: effect, Z: conf}",,We know that {X_string} has an effect on {Y_st...,We observe that the probability of {Y_1} given...,"do_calculus; counterfactual_formal_solver,",,"[greater than, smaller than]",[opposite of relation1],text


In [ ]:
# confounder graph given sufficient text info 

with open('conf_id.json', 'w') as out_file:
  out_file.write("[\n")
  # Counter for number of entries written
  count = 0
  for idx, row in dat.iterrows():
    if row['causal_graph_type'] == 'confounder' and row['info_type'] == 'text':
      # loop over 2 different Structural equations
      for rel1 in row['relation1']:    
        if rel1 == 'greater than':
          rel2 = 'smaller than'
        else:
          rel2 = 'greater than'

        filled_template = row['given_info_word_template'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                                                X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1']) +' '+\
                          row['extra_info'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                                  X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                  rel1 = rel1, rel2 = rel2)
        
        # loop over all applicible queries with no observation (do_calculus only)
        for index, question in dat_query.iterrows():
          if 'confounder' in question['causal_graph_type'] and 'text' in question['info_type'] and question['CI_engine to get the answer'] == 'do_calculus': 
            if question['query_id'] == 'adj':
              query = question['query_word_form'].format(X_string = row['X_string_name'], Z_string = row['Z_string_name'], Y_string=row['Y_string_name'], W_string=row['W_string_name'], 
                                                                    X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                    W_0 = row['W_0'], W_1 = row['W_1'])

              target = row['Z_string_name']
             
            elif question['query_id'] == 'ace_pos':
              query = question['query_word_form'].format(X_string = row['X_string_name'], Z_string = row['Z_string_name'], Y_string=row['Y_string_name'], W_string=row['W_string_name'], 
                                                                    X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                                    W_0 = row['W_0'], W_1 = row['W_1'])
              
              if rel2 == 'greater than':
                target = f"Yes, {row['X_1']} would increase one's chance of {row['Y_1']}"
              else:
                target = f"No, {row['X_1']} would decrease one's chance of {row['Y_1']}"
              


            if count > 0:
              out_file.write(",\n")
                  
            data = {
                  "causal_graph_type": row['causal_graph_type'],
                  "causal_inference_engine": question['CI_engine to get the answer'],
                  "given_info_type": row['info_type'],
                  "input": filled_template + ' ' + query.replace('\n', ' '),
                  "target": target
              }




            # Write data to file
            json.dump(data, out_file, indent=4)
            # Increment counter
            count += 1
          

          elif 'confounder' in question['causal_graph_type'] and 'text' in question['info_type'] and question['CI_engine to get the answer'] == 'counterfactual': 
            # With observations, answer counterfactual questions.
            s = {'X','Z','Y'}
            powerset = set(frozenset(subset) for subset in itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1)))
            powerset = [set(subset) for subset in powerset]
            filled_template = row['given_info_word_template'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                                              X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                              rel1 = rel1)+' '+\
                              row['extra_info'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                                      X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'], 
                                                      rel1 = rel1, rel2 = rel2)


            for obs_set in powerset:
              n = len(obs_set)
              if n > 0:
                for obs_val in list(itertools.product(*[(0, 1)] * (n))):

                  evid_text = ""
                  evid = {}
                  for i, (k, v) in enumerate(zip(obs_set, obs_val)):
                    evid_text += row[f'{k}_{v}']
                    evid[k] = v
                    if i != len(obs_set) - 1:
                      evid_text += ', '

                  for cou_val in [0,1]:
                    if 'X' not in evid or cou_val != evid['X']:
                      query = question['query_word_form'].format(X_string = row['X_string_name'],  Y_1 = row['Y_1'],
                                                                X_cou_val = row[f'X_{cou_val}'], val_original = row[f'X_{abs(1-cou_val)}'], evidence = evid_text)
                      
            
                      if (rel2 == 'greater than' and cou_val == 1) or (rel2 == 'smaller than' and cou_val == 0):
                        target = f"Yes, {row[f'X_{cou_val}']} would have increased one's chance of {row['Y_1']}"
                      elif (rel2 == 'greater than' and cou_val == 0) or (rel2 == 'smaller than' and cou_val == 1):
                        target = f"No, {row[f'X_{cou_val}']} would have decreased one's chance of {row['Y_1']}"
                            
                      if count > 0:
                        out_file.write(",\n")


                      data = {
                            "causal_graph_type": row['causal_graph_type'],
                            "causal_inference_engine": question['CI_engine to get the answer'],
                            "given_info_type": row['info_type'],
                            "input": filled_template + ' ' + query.replace('\n', ' '),
                            "target": target
                        }
                      # Write data to file
                      json.dump(data, out_file, indent=4)
                      # Increment counter
                      count += 1             
                    

  out_file.write("\n]")
out_file.close()


In [ ]:
with open('conf_id.json', 'r') as f:
  unique_entries = {}
  duplicates = []
  data = json.load(f)
  print(f"Total number of data points in the file: {len(data)}")


  for entry in data:
      entry_hash = hash(json.dumps(entry, sort_keys=True))
      if entry_hash in unique_entries:
          duplicates.append(entry)
          print("Duplicate Entry:")
          print(json.dumps(entry, indent=2))
          print("Original Entry:")
          print(json.dumps(unique_entries[entry_hash], indent=2))
          print("-------------------")
      else:
          unique_entries[entry_hash] = entry

  if duplicates:
      print(f"There are {len(duplicates)} duplicates in the file.")
      print(f"Total number of data points in the file: {len(data)}")
      # Do something with the duplicates, such as printing them or removing them from the original data
  else:
      print("There are no duplicates in the file.")

out_file.close()

Total number of data points in the file: 216
There are no duplicates in the file.


In [ ]:
# Collider
# Share the same graphs as mediator2 and fork. In addition, we have examples from 'collider_>' and colider_m2'

with open('collider_>.json', 'w') as out_file:
  out_file.write("[\n")
  # Counter for number of entries written
  count = 0
  for idx, row in dat.iterrows():
    if row['causal_graph_type'] == 'collider_>' and row['info_type'] in ['text', 'monotonic']:
      filled_template = row['given_info_word_template'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                                            X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1']) +' '+\
                      row['extra_info'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                              X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'])
      
      # loop over all applicible queries with no observation (do_calculus only)
      for index, question in dat_query.iterrows():
        if 'collider_>' in question['causal_graph_type'] and row['info_type'] in question['info_type']: 
          if question['query_id'] == 'att':
            # With observations, answer counterfactual questions.
            s = {'X','Z','Y'}
            powerset = set(frozenset(subset) for subset in itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1)))
            powerset = [set(subset) for subset in powerset]

            for obs_set in powerset:
              n = len(obs_set)
              if 'X' in obs_set:
                for obs_val in list(itertools.product(*[(0, 1)] * (n-1))):
                  obs_val = (1,) + obs_val
                  evid_text = ""
                  evid = {}
                  for i, (k, v) in enumerate(zip(obs_set, obs_val)):
                    evid_text += row[f'{k}_{v}']
                    evid[k] = v
                    if i != len(obs_set) - 1:
                      evid_text += ', '

                  query = question['query_word_form'].format(X_string = row['X_string_name'],  Y_1 = row['Y_1'],
                                                            X_0 = row['X_0'], evidence = evid_text, X_1 = row['X_1'])
                  
                  target = f"Yes, their chance of {row['Y_1']} would have decreased if their {row['X_string_name']} had been {row['X_0']}"

                      
                  if count > 0:
                    out_file.write(",\n")


                  data = {
                        "causal_graph_type": row['causal_graph_type'],
                        "causal_inference_engine": question['CI_engine to get the answer'],
                        "given_info_type": row['info_type'],
                        "input": filled_template + ' ' + query.replace('\n', ' '),
                        "target": target
                    }
                  # Write data to file
                  json.dump(data, out_file, indent=4)
                  # Increment counter
                  count += 1   

          else:
            if question['query_id'] == 'collider_bias':
              for (val, val1) in zip([0,1],[0,1]):
                val2 = abs(1-val1)
                query = question['query_word_form'].format(X_string = row['X_string_name'], Z_string = row['Z_string_name'], Y_string=row['Y_string_name'], 
                                                                      X_val1 = row[f'X_{val1}'], Y_val = row[f'Y_{val}'], Z_val2 = row[f'Z_{val2}'])

                target = f"No, {row['X_string_name']} is independent of {row['Z_string_name']}"
                if count > 0:
                  out_file.write(",\n")


                data = {
                      "causal_graph_type": row['causal_graph_type'],
                      "causal_inference_engine": question['CI_engine to get the answer'],
                      "given_info_type": row['info_type'],
                      'query_id': question['query_id'],
                      "input": filled_template + ' ' + query.replace('\n', ' '),
                      "target": target
                  }
                # Write data to file
                json.dump(data, out_file, indent=4)
                # Increment counter
                count += 1            
        
            elif question['query_id'] in ['nde_pos']:
              query = question['query_word_form'].format(X_string = row['X_string_name'],  Z_string = row['Z_string_name'], Y_1 = row['Y_1'],
                                            X_0 = row['X_0'], evidence = evid_text, X_1 = row['X_1'])
                

              target = f"Yes, {row['X_1']} must have a positive direct effect on the chance of {row['Y_1']}"
              if count > 0:
                out_file.write(",\n")


              data = {
                    "causal_graph_type": row['causal_graph_type'],
                    "causal_inference_engine": question['CI_engine to get the answer'],
                    "given_info_type": row['info_type'],
                    'query_id': question['query_id'],
                    "input": filled_template + ' ' + query.replace('\n', ' '),
                    "target": target
                }
              # Write data to file
              json.dump(data, out_file, indent=4)
              # Increment counter
              count += 1   

            elif question['query_id'] in ['nie_polarity']:
              query = question['query_word_form'].format(X_string = row['X_string_name'],  Z_string = row['Z_string_name'], Y_1 = row['Y_1'],
                              X_0 = row['X_0'], evidence = evid_text, X_1 = row['X_1'])
              if row['causal_graph_type'] == 'collider_>':
                target = f"No, {row['X_string_name']} is independent of {row['Z_string_name']}"

              else: 
                target = f"Yes, {row['X_1']} must have a positive direct effect on the chance of {row['Y_1']}"    

              if count > 0:
                out_file.write(",\n")


              data = {
                    "causal_graph_type": row['causal_graph_type'],
                    "causal_inference_engine": question['CI_engine to get the answer'],
                    "given_info_type": row['info_type'],
                    'query_id': question['query_id'],
                    "input": filled_template + ' ' + query.replace('\n', ' '),
                    "target": target
                }



              # Write data to file
              json.dump(data, out_file, indent=4)
              # Increment counter
              count += 1


    if row['causal_graph_type'] == 'mediator1' and row['info_type'] in ['text', 'monotonic']:
      filled_template = row['given_info_word_template'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                                          X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1']) +' '+\
                    row['extra_info'].format(X_string=row['X_string_name'], Y_string=row['Y_string_name'], Z_string=row['Z_string_name'],
                                            X_0 = row['X_0'], X_1 = row['X_1'], Y_0 = row['Y_0'], Y_1 = row['Y_1'], Z_0 = row['Z_0'], Z_1 = row['Z_1'])
      
      # loop over all applicible queries with no observation (do_calculus only)
      for index, question in dat_query.iterrows():
        if 'collider_>' in question['causal_graph_type'] and row['info_type'] in question['info_type']: 
          if question['query_id'] == 'att':
            # With observations, answer counterfactual questions.
            s = {'X','Z','Y'}
            powerset = set(frozenset(subset) for subset in itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1)))
            powerset = [set(subset) for subset in powerset]

            for obs_set in powerset:
              n = len(obs_set)
              if 'X' in obs_set:
                for obs_val in list(itertools.product(*[(0, 1)] * (n-1))):
                  obs_val = (1,) + obs_val
                  evid_text = ""
                  evid = {}
                  for i, (k, v) in enumerate(zip(obs_set, obs_val)):
                    evid_text += row[f'{k}_{v}']
                    evid[k] = v
                    if i != len(obs_set) - 1:
                      evid_text += ', '

                  query = question['query_word_form'].format(X_string = row['X_string_name'],  Y_1 = row['Y_1'],
                                                            X_0 = row['X_0'], evidence = evid_text, X_1 = row['X_1'])
                  
                  target = f"Yes, their chance of {row['Y_1']} would have decreased if their {row['X_string_name']} had been {row['X_0']}"

                      
                  if count > 0:
                    out_file.write(",\n")


                  data = {
                        "causal_graph_type": row['causal_graph_type'],
                        "causal_inference_engine": question['CI_engine to get the answer'],
                        "given_info_type": row['info_type'],
                        "input": filled_template + ' ' + query.replace('\n', ' '),
                        "target": target
                    }
                  # Write data to file
                  json.dump(data, out_file, indent=4)
                  # Increment counter
                  count += 1   

          else:
            if question['query_id'] == 'collider_bias':
              for (val, val1) in zip([0,1],[0,1]):
                val2 = abs(1-val1)
                query = question['query_word_form'].format(X_string = row['X_string_name'], Z_string = row['Z_string_name'], Y_string=row['Y_string_name'], 
                                                                      X_val1 = row[f'X_{val1}'], Y_val = row[f'Y_{val}'], Z_val2 = row[f'Z_{val2}'])

                target = f"No, {row['X_string_name']} is independent of {row['Z_string_name']}"
                if count > 0:
                  out_file.write(",\n")


                data = {
                      "causal_graph_type": row['causal_graph_type'],
                      "causal_inference_engine": question['CI_engine to get the answer'],
                      "given_info_type": row['info_type'],
                      'query_id': question['query_id'],
                      "input": filled_template + ' ' + query.replace('\n', ' '),
                      "target": target
                  }
                # Write data to file
                json.dump(data, out_file, indent=4)
                # Increment counter
                count += 1            
        
            elif question['query_id'] in ['nde_pos']:
              query = question['query_word_form'].format(X_string = row['X_string_name'],  Z_string = row['Z_string_name'], Y_1 = row['Y_1'],
                                            X_0 = row['X_0'], evidence = evid_text, X_1 = row['X_1'])
                

              target = f"Yes, {row['X_1']} must have a positive direct effect on the chance of {row['Y_1']}"
              if count > 0:
                out_file.write(",\n")


              data = {
                    "causal_graph_type": row['causal_graph_type'],
                    "causal_inference_engine": question['CI_engine to get the answer'],
                    "given_info_type": row['info_type'],
                    'query_id': question['query_id'],
                    "input": filled_template + ' ' + query.replace('\n', ' '),
                    "target": target
                }
              # Write data to file
              json.dump(data, out_file, indent=4)
              # Increment counter
              count += 1   

            elif question['query_id'] in ['nie_polarity']:
              query = question['query_word_form'].format(X_string = row['X_string_name'],  Z_string = row['Z_string_name'], Y_1 = row['Y_1'],
                              X_0 = row['X_0'], evidence = evid_text, X_1 = row['X_1'])
              if row['causal_graph_type'] == 'collider_>':
                target = f"No, {row['X_string_name']} is independent of {row['Z_string_name']}"

              else: 
                target = f"Yes, {row['X_1']} must have a positive direct effect on the chance of {row['Y_1']}"    

              if count > 0:
                out_file.write(",\n")


              data = {
                    "causal_graph_type": row['causal_graph_type'],
                    "causal_inference_engine": question['CI_engine to get the answer'],
                    "given_info_type": row['info_type'],
                    'query_id': question['query_id'],
                    "input": filled_template + ' ' + query.replace('\n', ' '),
                    "target": target
                }



              # Write data to file
              json.dump(data, out_file, indent=4)
              # Increment counter
              count += 1




                                

  out_file.write("\n]")
out_file.close()


In [ ]:
with open('conf_id.json', 'r') as f:
  unique_entries = {}
  duplicates = []
  data = json.load(f)
  print(f"Total number of data points in the file: {len(data)}")


  for entry in data:
      entry_hash = hash(json.dumps(entry, sort_keys=True))
      if entry_hash in unique_entries:
          duplicates.append(entry)
          print("Duplicate Entry:")
          print(json.dumps(entry, indent=2))
          print("Original Entry:")
          print(json.dumps(unique_entries[entry_hash], indent=2))
          print("-------------------")
      else:
          unique_entries[entry_hash] = entry

  if duplicates:
      print(f"There are {len(duplicates)} duplicates in the file.")
      print(f"Total number of data points in the file: {len(data)}")
      # Do something with the duplicates, such as printing them or removing them from the original data
  else:
      print("There are no duplicates in the file.")

out_file.close()

Total number of data points in the file: 216
There are no duplicates in the file.
